In [1]:
import nltk
from nltk.tokenize import word_tokenize
import re

In [2]:
def eliminate_implication(expression):
    p = expression.split("->")
    return expression.replace("->", " v ").replace(p[0] or p[1],"~" + p[0] or p[1])


In [12]:
test = "(p v q) -> (~p ∨ q)"
eliminate_implication(test)

'~(p v q)  v  (~p ∨ q)'

In [4]:
def move_negation_inward(sentence):
    tokens = word_tokenize(sentence)
    transformed_tokens = []
    i = 0
    while i < len(tokens):
        token = tokens[i]
        if token == '~':
            # Check if the next token is a left parenthesis indicating the start of a sub-expression
            if i < len(tokens) - 1 and tokens[i + 1] == '(':
                # Find the corresponding right parenthesis to the left parenthesis
                left_parenthesis_count = 1
                j = i + 2
                while j < len(tokens):
                    if tokens[j] == '(':
                        left_parenthesis_count += 1
                    elif tokens[j] == ')':
                        left_parenthesis_count -= 1
                    if left_parenthesis_count == 0:
                        break
                    j += 1
                
                # Move the negation inward by applying De Morgan's law
                sub_expression = ' '.join(tokens[i + 2:j])
                sub_tokens = word_tokenize(sub_expression)
                transformed_sub_tokens = []
                prev_token_was_negation = False
                for token in sub_tokens:
                    if token == '~':
                        prev_token_was_negation = True
                    elif token == '∧':
                        if prev_token_was_negation:
                            prev_token_was_negation = False
                            continue  # Skip appending the previous '~'
                        transformed_sub_tokens.append('v')
                    elif token == 'v':
                        if prev_token_was_negation:
                            prev_token_was_negation = False
                            continue  # Skip appending the previous '~'
                        transformed_sub_tokens.append('∧')
                    else:
                        if prev_token_was_negation:
                            transformed_sub_tokens.append(token)
                            prev_token_was_negation = False
                        else:
                            transformed_sub_tokens.append('~' + token)
                
                transformed_tokens.extend(transformed_sub_tokens)
                i = j + 1
            else:
                transformed_tokens.append(token)
                i += 1
        else:
            transformed_tokens.append(token)
            i += 1
    transformed_sentence = ' '.join(transformed_tokens)
    return transformed_sentence

sentence = "~(p v q)  v  (~p v q)"
transformed_sentence = move_negation_inward(sentence)
print("Original sentence:", sentence)
print("Transformed sentence:", transformed_sentence)

Original sentence: ~(p v q)  v  (~p v q)
Transformed sentence: ~p ∧ ~q v ( ~p v q )


In [5]:
def remove_double_not(expression):
    return expression.replace("~~", "")

In [6]:
s = "~~~~p"
remove_double_not(s)

'p'

In [29]:
def standardize_variable_Scope(Expressions):
    exp = Expressions.split("', '")
    standardizedExpressions = []
    for expression in exp:
        new_expr = expression
        variables = set()
        replacement_map = {}
        quantifier_found = False
        quantifier_var = None
        quantifier_replacement = None
        def new_variable(old_var):
            if old_var in replacement_map:
                return replacement_map[old_var]
            NEW_var = chr(ord('a') + len(variables))
            while NEW_var in variables:
                NEW_var = chr(ord(NEW_var) + 1)
            variables.add(NEW_var)
            replacement_map[old_var] = NEW_var
            return NEW_var
        i = 0
        while i < len(new_expr):
            if new_expr[i] in "∀∃":
                quantifier = new_expr[i]
                var = new_expr[i+1]
                if var.islower():
                    if not quantifier_found:
                        quantifier_found = True
                        quantifier_var = var
                        uantifier_replacement = var
                    else:
                        NEW_var = new_variable(var)
                        new_expr = new_expr[:i+1] + new_expr[i+1:].replace(var, NEW_var)
                        i += len(NEW_var) - 1
            i += 1
        standardizedExpressions.append(new_expr)
    return "', '".join(standardizedExpressions)
test = "(∀x P( x)) ∨ (∃x Q( x)) v (∀x Q( x))"
print(standardize_variable_Scope(test))

(∀x P( x)) ∨ (∃a Q( a)) v (∀b Q( b))


In [8]:
def prenex_form(expression):
    quantifiers = {'∀', '∃'}
    quantifier_part = ""
    rest_part = ""
    quantifier_buffer = ""
    for char in expression:
        if char in quantifiers:
            quantifier_buffer += char
        elif char == '(' and quantifier_buffer:
            quantifier_part += quantifier_buffer + ' '
            quantifier_buffer = ""
        elif char.isalpha() and quantifier_buffer:
            quantifier_part += quantifier_buffer + char + ' '
            quantifier_buffer = ""
        else:
            rest_part += char

    return quantifier_part + rest_part.strip()

expression = "(∀x P(x)) ∨ (∃y Q(y)) ∧ (∃z Q(z))"
print(prenex_form(expression))  # Output: ∀x ∃y P( x) ∨ Q( y)

∀x ∃y ∃z ( P(x)) ∨ ( Q(y)) ∧ ( Q(z))


In [39]:
def Skolemization(expression, universalVars=None, skolemFuncs=None, usedVars=None):
    if universalVars is None:
        universalVars = []
    if skolemFuncs is None:
        skolemFuncs = {}
    if usedVars is None:
        usedVars = set() 

    i = 0
    new_expression = ""
    default = None  
    while i < len(expression):
        if expression[i] in ["∀", "∃"]:
            Quantf = expression[i]
            var = expression[i + 1]
            if Quantf == "∀":
                universalVars.append(var)
                if default is None:  
                    default = var
                # Include ∀ quantifiers and variables in the new formula
                new_expression += f"{Quantf}{var} "
            else:  # quantifier == "∃"
                if universalVars:
                    # Use the first unused universal variable for the Skolem function, if available
                    for u_var in universalVars:
                        if u_var not in usedVars:
                            skolem_term = f"f({u_var})"
                            usedVars.add(u_var)
                            break
                    else:

                        skolem_term = f"f({default})"
                else:
                    skolem_term = "c" 
                skolemFuncs[var] = skolem_term
            i += 2  # Skip past the quantifier and variable
            continue
        elif expression[i] in skolemFuncs:
        
            var = expression[i]
            skolem_term = skolemFuncs[var]
            new_expression += skolem_term
            i += 1  # Adjust the index after replacement
        else:
            # Directly add other characters to the new formula
            new_expression += expression[i]
            i += 1

    return new_expression
str1 = "∀x ∃y ∃z P(x) ∨ Q(y) v S(z)"
print (Skolemization(str1))

∀x    P(x) ∨ Q(f(x)) v S(f(x))


In [43]:
# def eliminate_universal_quantifiers(expression):# like ∃x or ∀x
#     quantifiers = {'∀', '∃'}
#     quantifier_part = ""
#     rest_part = ""
#     quantifier_buffer = ""
#     for char in expression:
#         if char in quantifiers:
#             quantifier_buffer += char
#         elif char == '(' and quantifier_buffer:
#             quantifier_part += quantifier_buffer + ' '
#             quantifier_buffer = ""
#         elif char.isalpha() and quantifier_buffer:
#             quantifier_part += quantifier_buffer + char + ' '
#             quantifier_buffer = ""
#         else:
#             rest_part += char

#     return rest_part.strip()


# expression = "∀x(∃yP(y) ∨ ∀u∃v(R(x, u) ∨ Q(x, v))"
# print(eliminate_universal_quantifiers(expression))  # Output: ∀x(P(f(x)) ∨ ∀u(R(x, u) ∨ Q(x, g(x, u))
def eliminate_universal_quantifiers(expression):
    return re.sub(r'∀\w+\s', '', expression)

expression = "∀x P(x) ∨ Q(F(x))"
print(eliminate_universal_quantifiers(expression))  

P(x) ∨ Q(F(x))


In [54]:
def TO_CNF(Expr):
    print("Before CNF: ", Expr)
   # Step 1
    s1 = eliminate_implication(Expr)
    # Step 2
    s2 = move_negation_inward(s1)
    # Step 3
    s3 = remove_double_not(s2)
    # Step 4
    s4 = standardize_variable_Scope(s3)
    # Step 5
    s5 = prenex_form(s4)
    # Step 6
    s6 = Skolemization(s5)
    # Step 7
    s7 = eliminate_universal_quantifiers(s6)
    return s7
        

In [69]:
STR2 = "∀x∃y∃z((l(x,y)∧l(y,z))∧(r(z)->p(z))∧(p(z)->r(z)))"
STR3 = "~~(p(a))"
STR4 = "∀x∀y∀z(f(x,y)->f(y,z))∧(f(x,z))"
STR5 = "∀x(∃y(p(y,d,b,c))∨q(d,b))∧p(x,y)->r(x)"
STR6 = "∀x∃y∃z((l(x,y)∧l(y,z))∧(r(z)->p(z))∧(p(z)->r(z)))"
STR7 = "∀x((~(p(a))∧(q(a))))"
STR8 = "∀x∀z∀u∀w((p(x,f(x),z))∨(p(u,w,w)))"
STR9 = "∀x∀a∀b((p(x,a,b)) ∨ (p(b,b,b)))"
print("1", TO_CNF(STR2))
print("2", TO_CNF(STR3))
print("3", TO_CNF(STR4))
print("4", TO_CNF(STR5))
print("5", TO_CNF(STR6))
print("6", TO_CNF(STR7))
print("7", TO_CNF(STR8))
print("8", TO_CNF(STR9))

Before CNF:  ∀x∃y∃z((l(x,y)∧l(y,z))∧(r(z)->p(z))∧(p(z)->r(z)))
1    ~ ( ( l ( x , f(x) ) ∧l ( f(x) , f(x) ) ) ∧ ( r ( f(x) ) v p ( f(x) ) ) ∧ ( p ( f(x) ) v r ( f(x) ) ) )
Before CNF:  ~~(p(a))
2 ~ ( p ( a ) )
Before CNF:  ∀x∀y∀z(f(x,y)->f(y,z))∧(f(x,z))
3    ~ ( f ( x , a ) v f ( a , b ) ) ∧ ( f ( x , b ) )
Before CNF:  ∀x(∃y(p(y,d,b,c))∨q(d,b))∧p(x,y)->r(x)
4   ~ (  ( p ( f(x) , d , b , c ) ) ∨q ( d , b ) ) ∧p ( x , f(x) ) v r ( x )
Before CNF:  ∀x∃y∃z((l(x,y)∧l(y,z))∧(r(z)->p(z))∧(p(z)->r(z)))
5    ~ ( ( l ( x , f(x) ) ∧l ( f(x) , f(x) ) ) ∧ ( r ( f(x) ) v p ( f(x) ) ) ∧ ( p ( f(x) ) v r ( f(x) ) ) )
Before CNF:  ∀x((~(p(a))∧(q(a))))
6  ~ ( ( ~p ~( ~a ~) ∧ ( q ( a ) ) ) )
Before CNF:  ∀x∀z∀u∀w((p(x,f(x),z))∨(p(u,w,w)))
7     ~ ( ( p ( x , f ( x ) , a ) ) ∨ ( p ( b , c , c ) ) )
Before CNF:  ∀x∀a∀b((p(x,a,b)) ∨ (p(b,b,b)))
8    ~ ( ( p ( x , a , b ) ) ∨ ( p ( b , b , b ) ) )


In [119]:
import sympy

def turn_conjunctions_into_clauses(cnf_expression):
    if isinstance(cnf_expression, sympy.And):
        clauses = set(cnf_expression.args)
    else:
        clauses = set([cnf_expression])
    
    # Replace logical OR symbol with comma
    clauses_str = [str(clause).replace('∨', ',') if '∧' not in str(clause) else str(clause).replace('∧', '}    {') for clause in clauses]
    
    return set(clauses_str)

In [127]:
expression = "∀x P(x) ∨ Q(F(x)) ∧ (Q(F(z))∨ Q(F(x))) "
turn_conjunctions_into_clauses(expression)

{'∀x P(x) ∨ Q(F(x)) }    { (Q(F(z))∨ Q(F(x))) '}

In [123]:
def rename_variables(clauses):
    unique_var_names = [sympy.Symbol(f'var{i}') for i in range(len(clauses))]
    substitutions = dict(zip(clauses, unique_var_names))
    renamed_clauses = [clause.subs(substitutions) for clause in clauses]
    return renamed_clauses